In [1]:
#Initial environement of computations

import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests


number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

#Moon equation form the Parisian Alfonsines Tables
Toledan_solarvel=HTable.read(1526, with_units=False)
# Historical target table
Hist_table=HTable.read(1374, with_units=False)
Hist_table=Hist_table.iloc[:180]



In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(180):
    Test_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)
    Delta_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)


In [8]:
with set_precision(tmode=TruncatureMode.ROUND, pmode=2):
    #Interpolate the recomputed table from the Toledan sources
    A=Toledan_solarvel.populate(list(Sexagesimal.range(180)), method="interpolate")

    for i in range(180):
        Test_recomp[i][1]=A[i+1][1]
        Delta_recomp[i][1]=3600*(Hist_table[i][1]-Test_recomp[i][1])
Delta_recomp

Moon argument of anomaly,Entries
Historical,Historical
01 ;,"00 ; 00,00"
02 ;,"00 ; 00,00"
03 ;,"01 ; 00,00"
04 ;,"01 ; 00,00"
05 ;,"01 ; 00,00"
06 ;,"00 ; 00,00"
07 ;,"01 ; 00,00"
08 ;,"01 ; 00,00"
09 ;,"01 ; 00,00"


In [11]:
# residuals visualisation and statistical summaries 1
B=Delta_recomp.to_pandas()
(B.mean().mean(), B.std().std())

(0.011142061281337047, nan)

In [12]:
B.astype(float).style.background_gradient(axis=None)


,Entries
Moon argument of anomaly,
01 ;,0.000000
02 ;,0.000000
03 ;,1.000000
04 ;,1.000000
05 ;,1.000000
06 ;,0.000000
07 ;,1.000000
08 ;,1.000000
09 ;,1.000000


In [13]:
#csv export of the recomputed table
C=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Lunar_velocity_TM.csv')
C.to_csv('Lunar_velocity_TM.zip', index=False, compression=compression_opts)